In [1]:
import os
from transformers import pipeline
import json
import numpy as np
import pandas as pd
VIDEOS = ['jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp',
 'JFBB_Rhythm_Section_Day1_Morning Thomas',
 'JFBB_Rhythm_Section_Day1_Afternoon Thomas',
 'JFBB_Jazz_Funk_morning_video20230419101818 Luis Cal García',
 'JFBB_Jazz_Funk_afternoon_video20230419143359 Luis Cal García',
 'jazzaar-20230417-afternoon-JFLB-Cuban-Vocals-video Philipp',
 'jazzaar-20230417-morning-JFLB-Cuban-Vocals-video Philipp']

/home/bsauter/anaconda3/envs/aaWorkshop/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-02 05:53:34.035988: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-02 05:53:36.092750: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-02 05:53:36.104344: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 05:53:42.863528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [37]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
columns = ['segment_id', 'start','end','sadness','joy','love','anger','fear','surprise']
dimensions = (-1,9)
for video in VIDEOS:
    output = []
    for root, dir, files in os.walk(f"../data/transcriptions/processed/{video}"):
        for file in files:
            
            with open(f'{root}/{file}') as f:
                text = f.read()
                f.close()

            prediction = classifier(text)
            emotions = [x['score'] for x in prediction[0]]
            meta_info = [file.split('_')[i] for i in range(3)]
            output = output + meta_info + emotions
            
    output = np.reshape(output, dimensions)
    output_df = pd.DataFrame(output, columns=columns)
    output_df= output_df.astype({'segment_id': 'int32'})
    output_df = output_df.sort_values(by=['segment_id']).reset_index(drop=True)  
    output_df.to_csv(f'../data/emotion_from_transcript/distilbert-base-uncased-emotion/{video}.csv')

Some layers from the model checkpoint at bhadresh-savani/distilbert-base-uncased-emotion were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at bhadresh-savani/distilbert-base-uncased-emotion and are newly initialized: ['dropout_279']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/bsaut

In [38]:
classifier = pipeline("text-classification",model='j-hartmann/emotion-english-distilroberta-base', return_all_scores=True)
columns = ['segment_id', 'start','end','anger','disgust','fear','joy','neutral','sadness', 'surprise']
dimensions = (-1,10)
for video in VIDEOS:
    output = []
    for root, dir, files in os.walk(f"../data/transcriptions/processed/{video}"):
        for file in files:
            
            with open(f'{root}/{file}') as f:
                text = f.read()
                f.close()

            prediction = classifier(text)
            emotions = [x['score'] for x in prediction[0]]
            meta_info = [file.split('_')[i] for i in range(3)]
            output = output + meta_info + emotions
            
    output = np.reshape(output, dimensions)
    output_df = pd.DataFrame(output, columns=columns)
    output_df= output_df.astype({'segment_id': 'int32'})
    output_df = output_df.sort_values(by=['segment_id']).reset_index(drop=True)
    output_df.to_csv(f'../data/emotion_from_transcript/emotion-english-distilroberta-base/{video}.csv')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/home/bsauter/anaconda3/envs/aaWorkshop/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
classifier = pipeline("sentiment-analysis")
columns = ['segment_id', 'start','end','positive', 'negative']
dimensions = (-1,5)
for video in VIDEOS:
    output = []
    for root, dir, files in os.walk(f"../data/transcriptions/processed/{video}"):
        for file in files:
            
            with open(f'{root}/{file}') as f:
                text = f.read()
                f.close()

            prediction = classifier(text)
            if prediction[0]['label'] == 'POSITIVE':
                emotions = [1,0]
            else:
                emotions = [0,1]

            meta_info = [file.split('_')[i] for i in range(3)]
            output = output + meta_info + emotions
            
    output = np.reshape(output, dimensions)
    output_df = pd.DataFrame(output, columns=columns)
    output_df= output_df.astype({'segment_id': 'int32'})
    output_df = output_df.sort_values(by=['segment_id']).reset_index(drop=True)
    output_df.to_csv(f'../data/emotion_from_transcript/binary_sentiment/{video}.csv')


from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998656511306763}]

In [3]:
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["No this does not work"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9997598528862}]